<a href="https://colab.research.google.com/github/vuongvmu/GCL_DemoCode/blob/main/socket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 25 14:18:28 2024

@author:
"""

import asyncio
import av
import cv2
import io
from multiprocessing import Process, Queue, Event
import time
import websockets
import socket
import time

def display_frame(frame, start_time, pts_offset, frame_rate):
    if frame.pts is not None:
        play_time = (frame.pts - pts_offset) * frame.time_base.numerator / frame.time_base.denominator
        if start_time is not None:
            current_time = time.time() - start_time
            time_diff = play_time - current_time
            if time_diff > 1 / frame_rate:
                return False
            if time_diff > 0:
                time.sleep(time_diff)
    img = frame.to_ndarray(format='bgr24')
    cv2.imshow('Video', img)
    return True

def get_pts(frame):
    return frame.pts

def render(terminated, data_queue):
    rawData = io.BytesIO()
    cur_pos = 0
    frames_buffer = []
    start_time = None
    pts_offset = None
    got_key_frame = False
    while not terminated.is_set():
        try:
            data = data_queue.get_nowait()#Lấy tất cả buff trong queue
        except:
            time.sleep(0.01)
            continue
        rawData.write(data) #lưu vào memorry
        rawData.seek(cur_pos)#Lấy con trỏ
        if cur_pos == 0:
            container = av.open(rawData, mode='r')# Mở dữ liệu giải mã
            original_codec_ctx = container.streams.video[0].codec_context# Lấy codec context
            codec = av.codec.CodecContext.create(original_codec_ctx.name, 'r')
        cur_pos += len(data)# tăng bị trí control
        dts = None
        for packet in container.demux():
            if packet.size == 0:
                continue
            dts = packet.dts
            if pts_offset is None:
                pts_offset = packet.pts
            if not got_key_frame and packet.is_keyframe:
                got_key_frame = True
            if data_queue.qsize() > 8 and not packet.is_keyframe:
                got_key_frame = False
                continue
            if not got_key_frame:
                continue
            frames = codec.decode(packet)
            if start_time is None:
                start_time = time.time()
            frames_buffer += frames
            frames_buffer.sort(key=get_pts)
            for frame in frames_buffer:
                if display_frame(frame, start_time, pts_offset, codec.framerate):
                    frames_buffer.remove(frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        if dts is not None:
            container.seek(25000)
        rawData.seek(cur_pos)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    terminated.set()
    cv2.destroyAllWindows()



async def receive_encoded_video(websocket, path):
    data_queue = Queue()
    terminated = Event()
    p = Process(
        target=render,
        args=(terminated, data_queue)
    )
    p.start()
    while not terminated.is_set():
        try:
            data = await websocket.recv()
        except:
            break
        data_queue.put(data)
    terminated.set()

#https://gist.github.com/tuxfight3r/1762636f788b14425c42f16d87dbc229
#https://gist.github.com/sunmeat/02b60c8a3eaef3b8a0fb3c249d8686fd
#!/usr/bin/python
'''
	udp socket client
	Silver Moon
'''

import socket	#for sockets
import sys	#for exit

# create dgram udp socket
try:
	s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
except socket.error:
	print ('Failed to create socket')
	sys.exit()

host = 'localhost';
port = 8888;

while(1) :
	msg = raw_input('Enter message to send : ')

	try :
		#Set the whole string
		s.sendto(msg, (host, port))

		# receive data from client (data, addr)
		d = s.recvfrom(1024)
		reply = d[0]
		addr = d[1]

		print ('Server reply : ' + reply)

	except socket.error, msg:
		print ('Error Code : ' + str(msg[0]) + ' Message ' + msg[1])
		sys.exit()

In [ ]:
import socket

UDP_IP = "127.0.0.1"
UDP_PORT = 5005

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, UDP_PORT))

while True:
    data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
    print("received message: %s" % data)




UDP_IP = "127.0.0.1"
UDP_PORT = 5005
MESSAGE = b"Hello, World!"

print("UDP target IP: %s" % UDP_IP)
print("UDP target port: %s" % UDP_PORT)
print("message: %s" % MESSAGE)

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))

while(1) :
	msg = "Hello"

	try :
		#Set the whole string
		sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))

		# receive data from client (data, addr)
		d = sock.recvfrom(1024)
		reply = d[0]
		addr = d[1]

		print ('Server reply : ' +str( reply))

	except (socket.error):
		print ('Error Code : ' + str(msg[0]) + ' Message ' + msg[1])
		sys.exit()

In [ ]:
# -*- coding: utf-8 -*-

import asyncio
import av
import cv2
import io
from multiprocessing import Process, Queue, Event
import time
import websockets
import socket
import time
import base64
import os


def display_frame(frame, start_time, pts_offset, frame_rate):
    if frame.pts is not None:
        play_time = (frame.pts - pts_offset) * frame.time_base.numerator / frame.time_base.denominator
        if start_time is not None:
            current_time = time.time() - start_time
            time_diff = play_time - current_time
            if time_diff > 1 / frame_rate:
                return False
            if time_diff > 0:
                time.sleep(time_diff)
    img = frame.to_ndarray(format='bgr24')
    cv2.imshow('Video', img)

    return True

def get_pts(frame):
    return frame.pts

def render(terminated, data_queue):
    rawData = io.BytesIO()
    cur_pos = 0
    frames_buffer = []
    start_time = None
    pts_offset = None
    got_key_frame = False
    while not terminated.is_set():
        try:
            data = data_queue.get(1024)#Lấy tất cả buff trong queue
        except:
            time.sleep(0.01)
            continue
        rawData.write(data) #lưu vào memorry
        rawData.seek(cur_pos)#Lấy con trỏ
        if cur_pos == 0:
            container = av.open(rawData, mode='r')# Mở dữ liệu giải mã
            original_codec_ctx = container.streams.video[0].codec_context# Lấy codec context
            codec = av.codec.CodecContext.create(original_codec_ctx.name, 'r')
        cur_pos += len(data)# tăng bị trí control
        dts = None
        for packet in container.demux():
            if packet.size == 0:
                continue
            dts = packet.dts
            if pts_offset is None:
                pts_offset = packet.pts
            if not got_key_frame and packet.is_keyframe:
                got_key_frame = True
            if data_queue.qsize() > 8 and not packet.is_keyframe:
                got_key_frame = False
                continue
            if not got_key_frame:
                continue
            frames = codec.decode(packet)
            if start_time is None:
                start_time = time.time()
            frames_buffer += frames
            frames_buffer.sort(key=get_pts)
            for frame in frames_buffer:
                if display_frame(frame, start_time, pts_offset, codec.framerate):
                    frames_buffer.remove(frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        if dts is not None:
            container.seek(25000)
        rawData.seek(cur_pos)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    terminated.set()
    cv2.destroyAllWindows()

def get_filepaths(directory):
    file_paths = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
    return file_paths


data_queue = Queue()
terminated = Event()

# async def receive_encoded_video(websocket, path):
#     data_queue = Queue()
#     terminated = Event()
#     with open(path, "rb") as videoFile:
#          base_text = base64.b64encode(videoFile.read())
#          data_queue = base64.b64decode(base_text)
#     p = Process(
#         target=render,
#         args=(terminated, data_queue)
#     )
#     p.start()


#     # while not terminated.is_set():
#     #     try:
#     #         data = await websocket.recv()
#     #     except:
#     #         break
#     #     data_queue.put(data)
#     # terminated.set()


# p = Process(
#      target=render,
#      args=(terminated, data_queue)
#  )
# p.start()
# for f in files:
#   path = os.path.normpath(f)
#   with open(path, "rb") as videoFile:
#        base_text = base64.b64encode(videoFile.read())
#        data_queue.put(base64.b64decode(base_text))

# def decodefromdic(video_dic):
#     data_queue = Queue()
#     terminated = Event()
#     files=get_filepaths(video_dic);

#     p = Process(
#          target=render,
#          args=(terminated, data_queue)
#      )
#     p.start()

#     for f in files:
#       path = os.path.normpath(f)
#       with open(path, "rb") as videoFile:
#            base_text = base64.b64encode(videoFile.read())
#            data_queue.put(base64.b64decode(base_text))
#     terminated.set()
video_path=r'F:\0.Data\3.Video\1.tfClassification\UCF101_videos\Split\val\BenchPress'
files=get_filepaths(video_path);

for f in files:
  path = os.path.normpath(f)
  with open(path, "rb") as videoFile:
       base_text = base64.b64encode(videoFile.read())
       data_queue.put(base64.b64decode(base_text))

render(terminated,data_queue)

# video_path=r'F:\0.Data\3.Video\1.tfClassification\UCF101_videos\Split\val\BenchPress\v_BenchPress_g04_c04.avi'
# with open(video_path, "rb") as videoFile:
#      base_text = base64.b64encode(videoFile.read())
#      data_queue.put( base64.b64decode(base_text))

# render(terminated,data_queue)

#https://gist.github.com/tuxfight3r/1762636f788b14425c42f16d87dbc229
#https://gist.github.com/sunmeat/02b60c8a3eaef3b8a0fb3c249d8686fd
#!/usr/bin/python
'''
	udp socket client
	Silver Moon
'''

# import socket	#for sockets
# import sys	#for exit

# # create dgram udp socket
# try:
# 	s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# except socket.error:
# 	print ('Failed to create socket')
# 	sys.exit()

# host = 'localhost';
# port = 8888;

# while(1) :
# 	msg = "Hello"

# 	try :
# 		#Set the whole string
# 		s.sendto(msg, (host, port))

# 		# receive data from client (data, addr)
# 		d = s.recvfrom(1024)
# 		reply = d[0]
# 		addr = d[1]

# 		print ('Server reply : ' +str( reply))

# 	except (socket.error):
# 		print ('Error Code : ' + str(msg[0]) + ' Message ' + msg[1])
# 		sys.exit()

# import socket

# UDP_IP = "127.0.0.1"
# UDP_PORT = 5005

# sock = socket.socket(socket.AF_INET, # Internet
#                      socket.SOCK_DGRAM) # UDP
# sock.bind((UDP_IP, UDP_PORT))

# while True:
#     data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
#     print("received message: %s" % data)



# import socket
# UDP_IP = "127.0.0.1"
# UDP_PORT = 5005
# MESSAGE = b"Hello, World!"

# print("UDP target IP: %s" % UDP_IP)
# print("UDP target port: %s" % UDP_PORT)
# print("message: %s" % MESSAGE)

# sock = socket.socket(socket.AF_INET, # Internet
#                      socket.SOCK_DGRAM) # UDP
# sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))

# while(1) :
# 	msg = "Hello"

# 	try :
# 		#Set the whole string
# 		sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))

# 		# receive data from client (data, addr)
# 		d = sock.recvfrom(1024)
# 		reply = d[0]
# 		addr = d[1]

# 		print ('Server reply : ' +str( reply))

# 	except (socket.error):
# 		print ('Error Code : ' + str(msg[0]) + ' Message ' + msg[1])
# 		sys.exit()



